In [1]:
import os 
import pandas as pd
import numpy as np 


In [30]:
from keras.datasets import imdb
import tensorflow as tf

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM,Dense, SpatialDropout1D, Dropout, SimpleRNN
from keras.layers.core import Dense, Activation, Dropout

In [3]:
pd.DataFrame(imdb.load_data())

,0,1
0,"[[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65,...","[1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, ..."
1,"[[1, 591, 202, 14, 31, 6, 717, 10, 10, 18142, ...","[0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, ..."


In [4]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data()

## EDA

In [5]:
train_data.shape, train_labels.shape, test_data.shape, test_labels.shape


((25000,), (25000,), (25000,), (25000,))

In [6]:
pd.DataFrame(train_data).head()

,0
0,"[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ..."
1,"[1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,..."
2,"[1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5..."
3,"[1, 4, 18609, 16085, 33, 2804, 4, 2040, 432, 1..."
4,"[1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1..."


In [7]:
pd.DataFrame(train_labels).head()

,0
0,1
1,0
2,0
3,1
4,0


imdb.get_word_index() # path="imdb_word_index.json")

In [9]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in train_data[0]] )
print(decoded,  "\n Review:",train_labels[0]) 


# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert redford's is an amazing actor and now the same being director norman's father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the part's of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be

In [54]:
len(imdb.get_word_index())

88584

In [44]:
vocab_size = len(imdb.get_word_index()) +1

In [45]:
max(len(i) for i in X_data)

2494

In [46]:


X_data =  pad_sequences(train_data, maxlen= 2494)

In [47]:
X_data.shape

(25000, 2494)

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, train_labels,
                                                    test_size=0.2, 
                                                    random_state=0)
 

In [49]:
X_train

array([[   0,    0,    0, ...,  675,  342,  158],
       [   0,    0,    0, ..., 7618,    9,  321],
       [   0,    0,    0, ...,  116,    5,   65],
       ...,
       [   0,    0,    0, ..., 3645,    7, 6644],
       [   0,    0,    0, ...,    4, 9114, 3796],
       [   0,    0,    0, ...,   46,    7,  158]], dtype=int32)

In [51]:
y_train

array([0, 1, 0, ..., 0, 0, 0])

In [52]:
X_train.shape, y_train.shape

((20000, 2494), (20000,))

In [55]:
model = Sequential()
model.add(Embedding(vocab_size, 2)) 
model.add(SimpleRNN(20)) # RNN layers 
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['acc'])

In [56]:
history = model.fit(X_train, y_train, 
                    epochs=4, 
                    batch_size=1000, 
                    validation_split=0.2)

/Users/reejungkim/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 16000 samples, validate on 4000 samples
Epoch 1/4
14000/16000 [=========================>....] - ETA: 9s - loss: 0.6931 - acc: 0.5161 

InvalidArgumentError:  indices[201,2444] = 88585 is not in [0, 88585)
	 [[node embedding_3/embedding_lookup (defined at /Users/reejungkim/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1751) ]] [Op:__inference_keras_scratch_graph_4417]

Function call stack:
keras_scratch_graph


In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(X_test, np.array(y_test))